In [1]:
from bs4 import BeautifulSoup
import requests
import json
import pandas as pd
import numpy as np
import re
import time

In [2]:
page_link = 'https://fr.wikipedia.org/wiki/Liste_des_communes_de_France_les_plus_peupl%C3%A9es#Communes_de_plus_de_30_000_habitants'
page_response = requests.get(page_link, timeout=5)
page_content = BeautifulSoup(page_response.content, "html.parser")
#print(page_content)

In [3]:
liste_ville = page_content.find("tbody").find_all("b")
liste_ville_net = []
liste_ville_href = []

for i in liste_ville:
    try :
        ville = i.find("a") 
        liste_ville_net.append(ville.text)
        liste_ville_href.append(ville['href'])
    except :
        pass
    

In [4]:
liste_ville_net[:50]

['Paris',
 'Marseille',
 'Lyon',
 'Toulouse',
 'Nice',
 'Nantes',
 'Montpellier',
 'Strasbourg',
 'Bordeaux',
 'Lille',
 'Rennes',
 'Reims',
 'Le Havre',
 'Saint-Étienne',
 'Toulon',
 'Grenoble',
 'Dijon',
 'Angers',
 'Nîmes',
 'Villeurbanne',
 'Saint-Denis',
 'Le Mans',
 'Aix-en-Provence',
 'Clermont-Ferrand',
 'Brest',
 'Tours',
 'Limoges',
 'Amiens',
 'Annecy',
 'Perpignan',
 'Boulogne-Billancourt',
 'Metz',
 'Besançon',
 'Orléans',
 'Saint-Denis',
 'Argenteuil',
 'Mulhouse',
 'Rouen',
 'Montreuil',
 'Caen',
 'Saint-Paul',
 'Nancy',
 'Nouméa',
 'Tourcoing',
 'Roubaix',
 'Nanterre',
 'Vitry-sur-Seine',
 'Avignon',
 'Créteil',
 'Dunkerque']

In [5]:
home_page = 'https://fr.wikipedia.org{}'
list_coor = []

for i in range(50):
    link = home_page.format(liste_ville_href[i])
    page_response = requests.get(link, timeout=5)
    page_content = BeautifulSoup(page_response.content, "html.parser")
    page_content
    coordinates = page_content.find("span", id="coordinates").find("a")
    cor_lon = coordinates["data-lon"]
    cor_lat = coordinates["data-lat"]
    list_coor.append(cor_lon + ',' + cor_lat)

In [6]:
list_coor

['2.351828,48.856578',
 '5.369889,43.296346',
 '4.842223,45.759723',
 '1.443962,43.604482',
 '7.271413,43.695949',
 '-1.55278,47.21806',
 '3.8772222222222,43.611944444444',
 '7.752353,48.573392',
 '-0.579541,44.837912',
 '3.063333,50.637222',
 '-1.679444,48.114722',
 '4.03472,49.26278',
 '0.1,49.49',
 '4.390278,45.434722',
 '5.93,43.1222',
 '5.726945,45.187778',
 '5.04139,47.32167',
 '-0.55416,47.47361',
 '4.36083333333,43.8377777778',
 '4.880278,45.766667',
 '55.4481,-20.8789',
 '0.196944,48.004167',
 '5.454025,43.531127',
 '3.082352,45.783088',
 '-4.485556,48.390834',
 '0.689167,47.393611',
 '1.25,45.85',
 '2.295736,49.894293',
 '6.133,45.916',
 '2.895556,42.698611',
 '2.24094672,48.8351979',
 '6.1778,49.1203',
 '6.02194,47.24306',
 '1.909,47.9025',
 '2.3539,48.9356',
 '2.2475,48.9478',
 '7.33994,47.749481',
 '1.099971,49.443232',
 '2.4431,48.8603',
 '-0.370556,49.182222',
 '55.2697,-21.0097',
 '6.1846,48.6936',
 '166.458,-22.2758',
 '3.1593,50.7217',
 '3.181666667,50.69',
 '2.2073,4

In [7]:
n = 50
m = 50
matrice = [0] * n
for i in range(n):
    matrice[i] = [0] * m
    
names = [_ for _ in liste_ville_net[:50]]
df = pd.DataFrame(matrice, index=names, columns=names)
df.head(5)

,Paris,Marseille,Lyon,Toulouse,Nice,Nantes,Montpellier,Strasbourg,Bordeaux,Lille,...,Saint-Paul,Nancy,Nouméa,Tourcoing,Roubaix,Nanterre,Vitry-sur-Seine,Avignon,Créteil,Dunkerque
Paris,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
Marseille,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
Lyon,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
Toulouse,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
Nice,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [15]:
for i in range(5) :
    for j in range(i+1, 5) :

        print(str(i) + " " + str(j))
        ville1=liste_ville_net[i]
        ville2=liste_ville_net[j]

        route = None
        while route is None:
            try:
                #print("try")
                time.sleep(1) 
                link = 'http://router.project-osrm.org/route/v1/driving/{};{}?overview=false'
                resp = requests.get(link.format(list_coor[i],list_coor[j]))
                content = json.loads(resp.content)
                route = content['routes']
            except:
                 pass
        df.loc[ville1,ville2] = route[0]['distance']
        

0 1
0 2
0 3
0 4
1 2
1 3
1 4
2 3
2 4
3 4


In [17]:
df.head(5)

,Paris,Marseille,Lyon,Toulouse,Nice,Nantes,Montpellier,Strasbourg,Bordeaux,Lille,...,Saint-Paul,Nancy,Nouméa,Tourcoing,Roubaix,Nanterre,Vitry-sur-Seine,Avignon,Créteil,Dunkerque
Paris,0,776537.1,466995.2,679873.8,933177.6,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
Marseille,0,0.0,313618.7,403432.2,198467.9,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
Lyon,0,0.0,0.0,557024.6,470237.1,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
Toulouse,0,0.0,0.0,0.0,562128.3,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
Nice,0,0.0,0.0,0.0,0.0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
